In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
import numpy as np
from src.helpers import Math, Image
from src import core
import cv2

In [ ]:
file = '../shared/predicted_smoothed.nrrd'

In [ ]:
detector = core.DetectLandmarks(file, overwrite=True)
arr= detector.arr
indices = detector.get_indices()
triangles_info = detector.get_triangles()
top_idx = indices['top_idx']
centroid_top = triangles_info['centroid_top']
midpoints_bottom = triangles_info['midpoints_bottom']
midpoints_top = triangles_info['midpoints_top']
img = np.copy(detector.arr[top_idx])
nx, ny = img.shape
lines_pixel_values_top = []
x_c, y_c = centroid_top
landmark_r_list = []
landmark_angle_list_top = []

In [ ]:
for midpoint in midpoints_top:
    line_pixel_values = []
    m = np.array(midpoint)
    c = np.array(centroid_top)
    x_axis = np.array([1, 0])
    cm = m - c
    angle = np.arctan2(cm[1], cm[0]) - np.arctan2(x_axis[1], x_axis[0])
    landmark_angle_list_top.append(angle)
    r = 0
    dr = 1
    x, y = centroid_top
    while (0 <= x < nx) and (0 <= y < ny):
        x = int(r * np.cos(angle) + x_c)
        y = int(r * np.sin(angle) + y_c)
        try:
            line_pixel_values.append(img[y, x])
            cv2.circle(img, (x, y), 10, color=2, thickness=-1)
            r += dr
        except:
            break
    lines_pixel_values_top.append(Math.pop_zeros(line_pixel_values))
    landmark_r_list.append(dr * len(line_pixel_values))
plt.imshow(img)

In [ ]:
top_landmark_coords_list = []
for i in range(3):
    r = landmark_r_list[i]
    angle = landmark_angle_list_top[i]
    x = int(r * np.cos(angle) + x_c)
    y = int(r * np.sin(angle) + y_c)
    top_landmark_coords_list.append((x, y))

In [ ]:
plt.imshow(img)
